## Imports and function decleration

In [1]:
#import modules
import sklearn as sk
import seaborn as sns
import numpy as np
import keras_tuner as kt
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import openpyxl

C:\Users\karlo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def get_best_n(df_input,model):
    scaler = StandardScaler()
    best = 0
    n_pca = 0
    scaler.fit(df_input)
    scaled_data = scaler.transform(df_input)
    for i, val in enumerate(range(len(scaled_data[1]))):
        if i == 0:
            i = 1
        pca = PCA(n_components = i)
        pca.fit(scaled_data)
        x_pca = pca.transform(scaled_data)
        nX_train, nX_test, ny_train, ny_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=123)
        n_model = model
        n_model = n_model.fit(nX_train, ny_train)
        val = n_model.score(nX_test,ny_test)
        if abs(val) > abs(best):
            best = val
            n_pca = i
    return (n_pca,best)

In [3]:
def best_pre_process(data,model,to_drop):
    proc_1 = get_best_n(data,model)
    val_1 = proc_1[1]
    df_proc_2 = data.drop(to_drop,axis=1)
    proc_2 = get_best_n(df_proc_2,model)
    val_2 = proc_2[1]
    if val_1 > val_2:
        return (proc_1[0],"Only PCA")
    else:
        return (proc_2[0],"Pairwise & PCA")

## Data preprocessing

In [4]:
df_regressionData = pd.read_excel("trainDataset.xls", usecols = ['RelapseFreeSurvival (outcome)','Age','original_shape_Elongation','original_shape_Flatness','original_shape_LeastAxisLength','original_shape_MajorAxisLength','original_shape_Maximum2DDiameterColumn','original_shape_Maximum2DDiameterRow','original_shape_Maximum2DDiameterSlice','original_shape_Maximum3DDiameter','original_shape_MeshVolume','original_shape_MinorAxisLength','original_shape_Sphericity','original_shape_SurfaceArea','original_shape_SurfaceVolumeRatio','original_shape_VoxelVolume','original_firstorder_10Percentile','original_firstorder_90Percentile','original_firstorder_Energy','original_firstorder_Entropy','original_firstorder_InterquartileRange','original_firstorder_Kurtosis','original_firstorder_Maximum','original_firstorder_MeanAbsoluteDeviation','original_firstorder_Mean','original_firstorder_Median','original_firstorder_Minimum','original_firstorder_Range','original_firstorder_RobustMeanAbsoluteDeviation','original_firstorder_RootMeanSquared','original_firstorder_Skewness','original_firstorder_TotalEnergy','original_firstorder_Uniformity','original_firstorder_Variance','original_glcm_Autocorrelation','original_glcm_ClusterProminence','original_glcm_ClusterShade','original_glcm_ClusterTendency','original_glcm_Contrast','original_glcm_Correlation','original_glcm_DifferenceAverage','original_glcm_DifferenceEntropy','original_glcm_DifferenceVariance','original_glcm_Id','original_glcm_Idm','original_glcm_Idmn','original_glcm_Idn','original_glcm_Imc1','original_glcm_Imc2','original_glcm_InverseVariance','original_glcm_JointAverage','original_glcm_JointEnergy','original_glcm_JointEntropy','original_glcm_MCC','original_glcm_MaximumProbability','original_glcm_SumAverage','original_glcm_SumEntropy','original_glcm_SumSquares','original_gldm_DependenceEntropy','original_gldm_DependenceNonUniformity','original_gldm_DependenceNonUniformityNormalized','original_gldm_DependenceVariance','original_gldm_GrayLevelNonUniformity','original_gldm_GrayLevelVariance','original_gldm_HighGrayLevelEmphasis','original_gldm_LargeDependenceEmphasis','original_gldm_LargeDependenceHighGrayLevelEmphasis','original_gldm_LargeDependenceLowGrayLevelEmphasis','original_gldm_LowGrayLevelEmphasis','original_gldm_SmallDependenceEmphasis','original_gldm_SmallDependenceHighGrayLevelEmphasis','original_gldm_SmallDependenceLowGrayLevelEmphasis','original_glrlm_GrayLevelNonUniformity','original_glrlm_GrayLevelNonUniformityNormalized','original_glrlm_GrayLevelVariance','original_glrlm_HighGrayLevelRunEmphasis','original_glrlm_LongRunEmphasis','original_glrlm_LongRunHighGrayLevelEmphasis','original_glrlm_LongRunLowGrayLevelEmphasis','original_glrlm_LowGrayLevelRunEmphasis','original_glrlm_RunEntropy','original_glrlm_RunLengthNonUniformity','original_glrlm_RunLengthNonUniformityNormalized','original_glrlm_RunPercentage','original_glrlm_RunVariance','original_glrlm_ShortRunEmphasis','original_glrlm_ShortRunHighGrayLevelEmphasis','original_glrlm_ShortRunLowGrayLevelEmphasis','original_glszm_GrayLevelNonUniformity','original_glszm_GrayLevelNonUniformityNormalized','original_glszm_GrayLevelVariance','original_glszm_HighGrayLevelZoneEmphasis','original_glszm_LargeAreaEmphasis','original_glszm_LargeAreaHighGrayLevelEmphasis','original_glszm_LargeAreaLowGrayLevelEmphasis','original_glszm_LowGrayLevelZoneEmphasis','original_glszm_SizeZoneNonUniformity','original_glszm_SizeZoneNonUniformityNormalized','original_glszm_SmallAreaEmphasis','original_glszm_SmallAreaHighGrayLevelEmphasis','original_glszm_SmallAreaLowGrayLevelEmphasis','original_glszm_ZoneEntropy','original_glszm_ZonePercentage','original_glszm_ZoneVariance','original_ngtdm_Busyness','original_ngtdm_Coarseness','original_ngtdm_Complexity','original_ngtdm_Contrast','original_ngtdm_Strength'])
df_result = df_regressionData.iloc[:, 0]
scaler = MinMaxScaler()
df_regressionData = pd.DataFrame(scaler.fit_transform(df_regressionData),columns=df_regressionData.columns,index=df_regressionData.index)
df_input = df_regressionData.iloc[:, 1:]

In [5]:
to_drop = ['original_shape_Maximum2DDiameterSlice',
            'original_shape_SurfaceArea',
            'original_firstorder_Energy',
            'original_firstorder_Mean',
            'original_firstorder_MeanAbsoluteDeviation',
            'original_firstorder_Uniformity',
            'original_firstorder_Entropy',
            'original_firstorder_Median', 
            'original_shape_VoxelVolume',
            'original_glcm_ClusterProminence',
            'original_glcm_ClusterTendency',
            'original_firstorder_Variance',
            'original_glcm_Contrast',
            'original_glcm_DifferenceAverage',
            'original_glcm_DifferenceVariance',
            'original_glcm_Id',
            'original_glcm_Idm',
            'original_glcm_Idmn',
            'original_glcm_DifferenceEntropy',
            'original_glcm_Idn',
            'original_glcm_JointEnergy',
            'original_glcm_JointEntropy',
            'original_glcm_SumEntropy',
            'original_glcm_SumSquares',
            'original_glcm_MaximumProbability',
            'original_gldm_LowGrayLevelEmphasis',
            'original_glrlm_GrayLevelNonUniformityNormalized',
            'original_gldm_HighGrayLevelEmphasis',
            'original_glrlm_GrayLevelNonUniformity',
            'original_glrlm_LongRunHighGrayLevelEmphasis',
            'original_gldm_GrayLevelNonUniformity',
            'original_gldm_GrayLevelVariance',
            'original_gldm_LargeDependenceEmphasis',
            'original_gldm_LargeDependenceHighGrayLevelEmphasis',
            'original_gldm_LargeDependenceLowGrayLevelEmphasis',
            'original_glrlm_GrayLevelVariance',
            'original_glszm_LowGrayLevelZoneEmphasis',
            'original_glrlm_ShortRunEmphasis',
            'original_glszm_SmallAreaLowGrayLevelEmphasis',
            'original_glrlm_LongRunEmphasis',
            'original_glszm_LargeAreaHighGrayLevelEmphasis',
            'original_ngtdm_Complexity',
            'original_ngtdm_Coarseness',
            'original_ngtdm_Contrast',
            'original_glcm_SumAverage'
            ]
df_corr_drop = df_input.drop(to_drop,axis=1)

In [6]:
result = best_pre_process(df_input,SVR(),to_drop)
result

(8, 'Pairwise & PCA')

In [7]:
pca = PCA(n_components = result[0])
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [8]:
x_pca.shape

(400, 8)

In [9]:
raw_test_data = pd.read_excel('testDatasetExample.xls',usecols = ['ID','Age','original_shape_Elongation','original_shape_Flatness','original_shape_LeastAxisLength','original_shape_MajorAxisLength','original_shape_Maximum2DDiameterColumn','original_shape_Maximum2DDiameterRow','original_shape_Maximum2DDiameterSlice','original_shape_Maximum3DDiameter','original_shape_MeshVolume','original_shape_MinorAxisLength','original_shape_Sphericity','original_shape_SurfaceArea','original_shape_SurfaceVolumeRatio','original_shape_VoxelVolume','original_firstorder_10Percentile','original_firstorder_90Percentile','original_firstorder_Energy','original_firstorder_Entropy','original_firstorder_InterquartileRange','original_firstorder_Kurtosis','original_firstorder_Maximum','original_firstorder_MeanAbsoluteDeviation','original_firstorder_Mean','original_firstorder_Median','original_firstorder_Minimum','original_firstorder_Range','original_firstorder_RobustMeanAbsoluteDeviation','original_firstorder_RootMeanSquared','original_firstorder_Skewness','original_firstorder_TotalEnergy','original_firstorder_Uniformity','original_firstorder_Variance','original_glcm_Autocorrelation','original_glcm_ClusterProminence','original_glcm_ClusterShade','original_glcm_ClusterTendency','original_glcm_Contrast','original_glcm_Correlation','original_glcm_DifferenceAverage','original_glcm_DifferenceEntropy','original_glcm_DifferenceVariance','original_glcm_Id','original_glcm_Idm','original_glcm_Idmn','original_glcm_Idn','original_glcm_Imc1','original_glcm_Imc2','original_glcm_InverseVariance','original_glcm_JointAverage','original_glcm_JointEnergy','original_glcm_JointEntropy','original_glcm_MCC','original_glcm_MaximumProbability','original_glcm_SumAverage','original_glcm_SumEntropy','original_glcm_SumSquares','original_gldm_DependenceEntropy','original_gldm_DependenceNonUniformity','original_gldm_DependenceNonUniformityNormalized','original_gldm_DependenceVariance','original_gldm_GrayLevelNonUniformity','original_gldm_GrayLevelVariance','original_gldm_HighGrayLevelEmphasis','original_gldm_LargeDependenceEmphasis','original_gldm_LargeDependenceHighGrayLevelEmphasis','original_gldm_LargeDependenceLowGrayLevelEmphasis','original_gldm_LowGrayLevelEmphasis','original_gldm_SmallDependenceEmphasis','original_gldm_SmallDependenceHighGrayLevelEmphasis','original_gldm_SmallDependenceLowGrayLevelEmphasis','original_glrlm_GrayLevelNonUniformity','original_glrlm_GrayLevelNonUniformityNormalized','original_glrlm_GrayLevelVariance','original_glrlm_HighGrayLevelRunEmphasis','original_glrlm_LongRunEmphasis','original_glrlm_LongRunHighGrayLevelEmphasis','original_glrlm_LongRunLowGrayLevelEmphasis','original_glrlm_LowGrayLevelRunEmphasis','original_glrlm_RunEntropy','original_glrlm_RunLengthNonUniformity','original_glrlm_RunLengthNonUniformityNormalized','original_glrlm_RunPercentage','original_glrlm_RunVariance','original_glrlm_ShortRunEmphasis','original_glrlm_ShortRunHighGrayLevelEmphasis','original_glrlm_ShortRunLowGrayLevelEmphasis','original_glszm_GrayLevelNonUniformity','original_glszm_GrayLevelNonUniformityNormalized','original_glszm_GrayLevelVariance','original_glszm_HighGrayLevelZoneEmphasis','original_glszm_LargeAreaEmphasis','original_glszm_LargeAreaHighGrayLevelEmphasis','original_glszm_LargeAreaLowGrayLevelEmphasis','original_glszm_LowGrayLevelZoneEmphasis','original_glszm_SizeZoneNonUniformity','original_glszm_SizeZoneNonUniformityNormalized','original_glszm_SmallAreaEmphasis','original_glszm_SmallAreaHighGrayLevelEmphasis','original_glszm_SmallAreaLowGrayLevelEmphasis','original_glszm_ZoneEntropy','original_glszm_ZonePercentage','original_glszm_ZoneVariance','original_ngtdm_Busyness','original_ngtdm_Coarseness','original_ngtdm_Complexity','original_ngtdm_Contrast','original_ngtdm_Strength'], sheet_name='Sheet1')
ID = raw_test_data['ID']
raw_test_data = raw_test_data.drop(['ID'],axis=1)
test_data = raw_test_data.drop(to_drop, axis=1)

In [10]:
test_data = pca.transform(test_data)

## Model and prediction

In [11]:
model = SVR(kernel = 'sigmoid', gamma = 'auto', coef0 = 1)

In [12]:
model_fit = model.fit(x_pca, df_result)

In [13]:
res = model_fit.predict(test_data)

In [14]:
dict_data = {'ID': ID,
             'Predicted RFS': res}
df_out = pd.DataFrame(dict_data)
df_out.to_excel('FinalTestXXX.xlsx', sheet_name='Regression', index=False)